# Selected Social Characteristics: Computers and Internet Use from the American Community Survey

**[Work in progress]**

This notebook downloads [selected social characteristics (DP02)](https://data.census.gov/cedsci/table?q=DP02&tid=ACSDP5Y2018.DP02) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-19636412-9e74-4bac-8a4c-c6c8b49bb9d3/installation-4.1.0/import


## Download selected variables

* [Selected social characteristics for US](https://data.census.gov/cedsci/table?q=DP02&tid=ACSDP5Y2018.DP02)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP02.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP02/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP02 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# COMPUTERS AND INTERNET USE
             'DP02_0150E': 'totalHouseholds',
             'DP02_0151E': 'withComputer',
             'DP02_0151PE': 'withComputerPct',
             'DP02_0152E': 'withBroadbandInternet',
             'DP02_0152PE': 'withBroadbandInternetPct',
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    if 'Pct' in v:
        print('c.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('c.' + v + ' = toInteger(row.' + v + '),')

c.totalHouseholds = toInteger(row.totalHouseholds),
c.withComputer = toInteger(row.withComputer),
c.withComputerPct = toFloat(row.withComputerPct),
c.withBroadbandInternet = toInteger(row.withBroadbandInternet),
c.withBroadbandInternetPct = toFloat(row.withBroadbandInternetPct),


## Download county-level data using US Census API

In [7]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [8]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6
0,DP02_0150E,DP02_0151E,DP02_0151PE,DP02_0152E,DP02_0152PE,state,county
1,18299,13858,75.7,11395,62.3,28,151
2,4563,3552,77.8,2943,64.5,28,111
3,3164,2345,74.1,1816,57.4,28,019
4,8706,7103,81.6,6093,70.0,28,057


##### Add column names

In [9]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [10]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [11]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [12]:
df.head()

,totalHouseholds,withComputer,withComputerPct,withBroadbandInternet,withBroadbandInternetPct,stateFips,countyFips
1,18299,13858,75.7,11395,62.3,28,151
2,4563,3552,77.8,2943,64.5,28,111
3,3164,2345,74.1,1816,57.4,28,019
4,8706,7103,81.6,6093,70.0,28,057
5,3658,2813,76.9,2276,62.2,28,015


In [13]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,totalHouseholds,withComputer,withComputerPct,withBroadbandInternet,withBroadbandInternetPct,stateFips,countyFips
1869,1118980,1051246,93.9,993171,88.8,06,073


In [14]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [15]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP02ComputersAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [16]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [17]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5
0,DP02_0150E,DP02_0151E,DP02_0151PE,DP02_0152E,DP02_0152PE,zip code tabulation area
1,3811,3047,80.0,2686,70.5,43964
2,19911,17999,90.4,16013,80.4,28216
3,26695,26117,97.8,25816,96.7,28277
4,9176,9045,98.6,8658,94.4,28278


##### Add column names

In [18]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('postalCode')
df.columns = columns

In [19]:
df.head()

,totalHouseholds,withComputer,withComputerPct,withBroadbandInternet,withBroadbandInternetPct,postalCode
1,3811,3047,80.0,2686,70.5,43964
2,19911,17999,90.4,16013,80.4,28216
3,26695,26117,97.8,25816,96.7,28277
4,9176,9045,98.6,8658,94.4,28278
5,13207,11601,87.8,10197,77.2,28303


In [20]:
# Example data
df.query("postalCode == '90210'")

,totalHouseholds,withComputer,withComputerPct,withBroadbandInternet,withBroadbandInternetPct,postalCode
30897,8036,7411,92.2,7150,89.0,90210


In [21]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [22]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP02ComputersZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [23]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [24]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [25]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [26]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [27]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,totalHouseholds,withComputer,withComputerPct,withBroadbandInternet,withBroadbandInternetPct,stateFips,countyFips,tract,source,aggregationLevel
56,2903,2883,99.3,2866,98.7,06,073,06073008324,American Community Survey 5 year,Tract
57,819,814,99.4,809,98.8,06,073,06073008339,American Community Survey 5 year,Tract
58,2180,2175,99.8,2056,94.3,06,073,06073008347,American Community Survey 5 year,Tract
59,3200,3186,99.6,3115,97.3,06,073,06073008354,American Community Survey 5 year,Tract
60,2116,1966,92.9,1878,88.8,06,073,06073008505,American Community Survey 5 year,Tract


### Save data

In [28]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP02ComputersTract.csv", index=False)

In [29]:
df.shape

(73056, 10)